In [5]:
#Importing packages:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import pi
import h5py
import sys

In [6]:
#####################################
#        GENERAL PARAMETERS         #
#####################################

    # Rcol for Hubble-flow calculation:
Mass=1e11
Gpc=4.3e-6

Ho = 70e-3  # [km/kpc/s] Hubble constant
lamM = 0.26  # Matter density 
lamL = 1-lamM # Dark Energy density
zinit = 15 # Starting density
zcol = 4 # Approximate collapse redshift (from linear theory)
ainit = 1./(1.+zinit) # a is the expansion factor
acol = 1./(1.+zcol)  # a is defined as 1/(1+z)
RHOcrit = 3.*(Ho**2)/(8.*pi*Gpc) # average density of Universe from Friedmann’s equation
Hinit = Ho*((lamM/ainit**3)-lamL)**0.5 # the value of Hubble constant at the initial redshift.
RHOcrit_init = (3*Hinit**2)/(8*pi*Gpc) # from Friedmann’s equation as above
LAMinit = 1.686*ainit/acol # the “overdensity” [(RHO-RHOave)/RHOave] required to collapse.
RHOcol = RHOcrit_init*(LAMinit+1) # alter RHOcrit_init by the amount suggested by linear theory
rcol = (3*Mass/(4*pi*RHOcol))**(1./3.) # radius of collapse... re-arranged Density = Mass/Vol

Mtotal = 1e11 #Total mass of DM halo
Rvir = (2*Gpc*Mtotal/(101*Ho**2))**(1/3)


In [7]:
#####################################
#         Parameter Arrays          #
#####################################

#PARAMETERS FOR ICs:
    #Number of particles: (1000 or 10000 particles)
npart = 1000

    #Particle distribution: ('sphere' for spherical uniform distribution,
    #                        'cubic' for cubic uniform distribution)
distrib = 'sphere'

    #Hubble flow: ('yes' for Vi=Hubble-Flow, 'no' for V0=0)
hubble = 'yes'

    #Rotation: ('yes' for initial rotation, 'no' for no initial rotation)
rotation = 'yes'

    #Final ICs file name and directory:
filename = 'ICs_hf_rot_n1000.hdf5'
icsdir = ''



#COORDINATES ARRAY:
if (distrib=='cube'):
        #Creating coordinates vector:
    long=0.9
    n=21
    N=n**3
    Mtotal=1
    m=Mtotal/N

    l1=np.linspace(0,long,n)
    l2=np.linspace(0,long,n)
    l3=np.linspace(0,long,n)

    x,y,z=np.meshgrid(l1,l2,l3)

    x.reshape(N,1)
    y.reshape(N,1)
    z.reshape(N,1)

    Coordinates = np.concatenate((x,y,z),axis = 1)
    Coordinates = Coordinates.reshape(N,3)
    print('(□ Cube): Particle distribution will be CUBICAL.')
elif (distrib=='sphere'):
        #Distribution:
    random.seed(77)
    Mtotal = 1e11 #10e4msun
    rmax = rcol #kpc
    Vtot = 4*np.pi*(rmax**3)/3 #kpc^3
    n = npart/(Vtot) #kpc^-3
    m = Mtotal/npart
    meanlen = (1/n)**(1/3) #kpc
    soft = meanlen/30
    sall = 1

    phi = np.random.uniform(0,2*np.pi,npart)
    costheta = np.random.uniform(-1,1,npart)
    u = np.random.uniform(0,1,npart)

    theta = np.arccos( costheta )
    r = rmax *u**(1/3)

    x = r * np.sin( theta) * np.cos( phi )
    y = r * np.sin( theta) * np.sin( phi )
    z = r * np.cos( theta )

        #Formatting:
    x.shape = (npart,1)
    y.shape = (npart,1)
    z.shape = (npart,1)
    Coordinates = np.concatenate((x,y,z),axis = 1)
    print('(◯ Sphere): Particle distribution will be SHERICAL.')
else:
    print('(!!!) Distribution option not valid, it MUST be (sphere) or (cube)\n      You wrote: '+distrib)
    




#IDs ARRAY:
ID = np.linspace(1,npart,npart,dtype = int)



#VELOCITIES ARRAY:
if (hubble=='no'):
    Velocities = np.zeros_like(Coordinates)
    print('(✗ Hubble flow): Hubble-flow NOT APPLIED to initial velocities. V0=0')
elif (hubble=='yes'):
    H=Hinit*(1.-LAMinit/3.)
    Hx,Hy,Hz=H*x,H*y,H*z
    Hx.shape = (npart,1)
    Hy.shape = (npart,1)
    Hz.shape = (npart,1)
    Velocities = np.concatenate((Hx,Hy,Hz),axis = 1)
    print('(✔ Hubble-flow): Hubble-flow APPLIED to initial velocities.')
else:
    print('(!!!) Rotation option not valid, it MUST be (yes) or (no)\n      You wrote: '+hubble)
    
    
#INITIAL ROTATION:
if (rotation=='yes'):
        # Initial rotation calculation:
    spin = 0.04 # Halo spin
    Vvir = np.sqrt((Gpc*Mtotal)/(Rvir)) # [km/s] Virial velocity
    J = np.sqrt(2) * spin * Mtotal * Vvir * Rvir # Total angular momentum
    Omega = J/(sum(r**2) * m) # Angular velocity
    vrot = Omega*r # [km/s] Rotation velocity
    vrot.shape = (npart,1)
    theta=np.arctan(y/x)

    for i in range(len(theta)):
        if theta[i]<0.0 and x[i]<0.0 or theta[i]>=0.0 and y[i]<0.0:
            theta[i]=theta[i]+np.pi

    vrot_x = np.sin(theta)*vrot
    vrot_y = -np.cos(theta)*vrot
    vrot_z = np.zeros(len(z))
    vrot_z.shape=(npart,1)

    #Velocity array caused by J:
    Angmom=np.concatenate((vrot_x,vrot_y,vrot_z),axis=1)
    Velocities=Velocities+Angmom

    print('(✔ Rotation): Rotation APPLIED to initial velocities.')
elif (rotation=='no'):
    print('(✗ Rotation): Rotation NOT APPLIED to initial velocities.')
else:
    print('(!!!) Rotation option not valid, it MUST be (yes) or (no)\n      You wrote: '+rotation)





#####################################
#           Creating hdf5           #
#####################################

#Creating File
file = h5py.File(icsdir+filename,'w')

#Creating Group
header = file.create_group("Header")
PartType1 = file.create_group("PartType1")
coordsDSET = file.create_dataset("/PartType1/Coordinates", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)
IDsDSET = file.create_dataset("/PartType1/ParticleIDs", (npart,), dtype =   h5py.h5t.STD_U32LE)
velsDSET = file.create_dataset("/PartType1/Velocities", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)

coordsDSET[...] = Coordinates
IDsDSET[...] = ID
velsDSET[...] = Velocities

nparts = np.zeros(3)
nparts[1] = npart
header.attrs['NumPart_ThisFile']    = nparts

massvec = np.zeros(3)
massvec[1] = Mtotal/npart

header.attrs['MassTable']           = massvec
header.attrs['Time']                = 0
header.attrs['Redshift']            = 15
header.attrs['NumPart_Total']       = nparts
header.attrs['NumFilesPerSnapshot'] = 1
header.attrs['BoxSize']             = 1.0
header.attrs['Omega0']              = 1.0
header.attrs['OmegaLambdda']        = 0.
header.attrs['HubbleParam']         = 0.7
header.attrs['Flag_Entropy_ICs']    = 0
header.attrs['NumPart_Total_HighWord'] = np.zeros(3)


# Closing file:
file.close()

(◯ Sphere): Particle distribution will be SHERICAL.
(✔ Hubble-flow): Hubble-flow APPLIED to initial velocities.
(✔ Rotation): Rotation APPLIED to initial velocities.
